### Задание №1
Скачайте текст "Литературных анекдотов". Напишите функцию, которая будет читать файл, лемматизировать текст с помощью pymystem3 и записывать результат в новый файл. У функции должно бы два аргумента: путь к исходному файлу и путь к файлу с лемматизированным текстом. Вызов функции тоже должен быть прописан в решении.


In [ ]:
условие если встречается - и перенос строки, джойнить их

### Задание №2
Очистите лемматизированный текст от стоп-слов и посчитайте ipm для оставшихся. Выведите 20 самых частотных по этому параметру слов.


### Задание №3
Сделайте полный морфологический разбор исходного текста. Напишите регулярное выражение, которое будет извлекать из тега только часть речи. Пройдитесь циклом по списку с разборами, который выдал pymystem3, извлекая из каждого разбора форму слова и его часть речи и записывая их в новый словарь (форма -- ключ, часть речи -- значение). Посчитайте абсолютную частоту для всех частей речи, а затем относительнную частоту (абсолютная частота / длина текста в словах).
